In [1]:
from fpl_helpers import *
import pandas as pd

In [2]:
elements_df, teams_df = get_tables()
global PLAYER_IDS
PLAYER_IDS = elements_df[['full_name', 'id']].set_index('full_name').to_dict()['id']
PLAYER_NAMES = {PLAYER_IDS[i]:i for i in list(PLAYER_IDS.keys()) }


In [3]:
def get_team(team_id, gw):
  url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/"
  r = requests.get(url)
  json = r.json()
  return json

team_ids = [i['element'] for i in get_team(715692, 10)['picks']]
team_names = [PLAYER_NAMES[i] for i in team_ids]

In [4]:
team_names

['David Raya Martin',
 'Pau Torres',
 'Rayan Aït-Nouri',
 'Gabriel dos Santos Magalhães',
 'Leif Davis',
 'Morgan Rogers',
 'Mohamed Salah',
 'Cole Palmer',
 'Liam Delap',
 'Chris Wood',
 'Erling Haaland',
 'Hákon Valdimarsson',
 'Taylor Harwood-Bellis',
 'Amadou Onana',
 'Yankuba Minteh']

In [5]:
compare_players(team_names[5:11], metrics=ATTACKER_METRICS, agg='mean')

,Difficulty,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected
Morgan Rogers,"5.0, 2.0, 4.0, 2.0, 2.0, 2.0",5.33,0.67,0.49,0.00,0.10,0.59,0.67,16.33,24.33,16.27,21.00,6.17,1.00,2397337.33
Mohamed Salah,"3.0, 2.0, 5.0, 4.0, 2.0, 2.0",10.33,1.00,0.57,0.33,0.11,0.68,2.33,35.33,46.27,21.13,41.33,10.87,1.00,4230326.67
Cole Palmer,"5.0, 2.0, 3.0, 2.0, 4.0, 2.0",5.00,0.33,0.22,0.00,0.24,0.46,1.33,22.67,18.80,25.33,17.67,6.20,1.00,5708525.33
Liam Delap,"4.0, 3.0, 2.0, 2.0, 2.0, 2.0",3.00,0.33,0.12,0.00,0.01,0.13,0.00,12.33,10.93,4.87,12.33,2.73,0.67,622487.00
Chris Wood,"3.0, 5.0, 2.0, 5.0, 3.0, 3.0",9.00,1.33,0.65,0.00,0.01,0.66,1.67,42.33,45.60,10.67,43.33,9.97,1.00,1656595.33
Erling Haaland,"3.0, 4.0, 5.0, 3.0, 2.0, 3.0",3.33,0.33,1.07,0.00,0.01,1.08,0.00,10.33,12.47,5.27,56.00,7.37,1.00,7778840.67


In [6]:
forwards = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==4)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
forwards['difficulty'] = forwards['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
forwards['avg_difficulty'] = forwards['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
forwards['cost_per_point'] = forwards['now_cost']/forwards['total_points']
#forwards[:20].sort_values(by='difficulty')
forwards

,full_name,team,selected_by_percent,now_cost,value_season,total_points,expected_goals,goals_scored,expected_assists,assists,expected_goals_conceded,goals_conceded,difficulty,avg_difficulty,cost_per_point
418,Erling Haaland,Man City,69.4,153,5.0,77,9.60,11,0.41,0,11.03,11,"[3.0, 4.0, 5.0, 3.0, 2.0, 3.0]",3.333333,1.987013
527,Chris Wood,Nott'm Forest,21.2,65,10.2,66,4.45,8,0.18,0,7.66,6,"[3.0, 5.0, 2.0, 5.0, 3.0, 3.0]",3.500000,0.984848
168,Danny Welbeck,Brighton,20.3,59,10.5,62,4.81,6,0.33,2,15.43,12,"[5.0, 3.0, 2.0, 3.0, 2.0, 2.0]",2.833333,0.951613
206,Nicolas Jackson,Chelsea,17.4,79,7.3,58,5.89,6,0.32,3,12.42,11,"[5.0, 2.0, 3.0, 2.0, 4.0, 2.0]",3.000000,1.362069
63,Ollie Watkins,Aston Villa,25.8,90,5.9,53,4.78,5,0.56,2,8.95,11,"[5.0, 2.0, 4.0, 2.0, 2.0, 2.0]",2.833333,1.698113
644,Matheus Santos Carneiro Da Cunha,Wolves,6.7,66,7.1,47,2.30,4,1.82,1,17.08,24,"[2.0, 3.0, 2.0, 2.0, 2.0, 2.0]",2.166667,1.404255
3,Kai Havertz,Arsenal,14.2,81,5.6,45,4.50,4,1.11,1,10.65,11,"[4.0, 3.0, 2.0, 3.0, 3.0, 2.0]",2.833333,1.800000
367,Jamie Vardy,Leicester,11.2,57,7.5,43,3.51,4,0.61,1,18.90,18,"[3.0, 4.0, 3.0, 2.0, 2.0, 4.0]",3.000000,1.325581
297,Raúl Jiménez,Fulham,11.0,58,7.4,43,3.44,4,1.12,1,7.03,8,"[2.0, 2.0, 4.0, 2.0, 5.0, 5.0]",3.333333,1.348837
668,Jørgen Strand Larsen,Wolves,2.4,56,7.7,43,2.48,4,0.45,1,16.23,23,"[2.0, 3.0, 2.0, 2.0, 2.0, 2.0]",2.166667,1.302326


In [7]:
forward_scores = compare_players(forwards['full_name'], metrics=ATTACKER_METRICS, agg='mean')
forward_scores.data['Average Difficulty'] = forward_scores.data['Difficulty'].apply(lambda x: np.mean(x))
forward_scores

,Difficulty,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty
Erling Haaland,"3.0, 4.0, 5.0, 3.0, 2.0, 3.0",3.33,0.33,1.07,0.00,0.01,1.08,0.00,10.33,12.47,5.27,56.00,7.37,1.00,7778840.67,3.333333
Chris Wood,"3.0, 5.0, 2.0, 5.0, 3.0, 3.0",9.00,1.33,0.65,0.00,0.01,0.66,1.67,42.33,45.60,10.67,43.33,9.97,1.00,1656595.33,3.500000
Danny Welbeck,"5.0, 3.0, 2.0, 3.0, 2.0, 2.0",7.00,0.67,0.44,0.33,0.01,0.45,1.33,27.67,29.93,10.70,27.67,6.83,1.00,2104154.67,2.833333
Nicolas Jackson,"5.0, 2.0, 3.0, 2.0, 4.0, 2.0",5.33,0.67,0.32,0.00,0.03,0.35,1.33,25.00,28.33,15.70,21.00,6.50,1.00,1832263.33,3.000000
Ollie Watkins,"5.0, 2.0, 4.0, 2.0, 2.0, 2.0",4.00,0.33,0.41,0.00,0.08,0.48,0.67,13.00,15.53,8.57,33.67,5.63,1.00,3048746.33,2.833333
Matheus Santos Carneiro Da Cunha,"2.0, 3.0, 2.0, 2.0, 2.0, 2.0",5.67,0.33,0.12,0.33,0.25,0.37,1.33,27.33,28.13,36.20,26.33,9.07,1.00,398022.67,2.166667
Kai Havertz,"4.0, 3.0, 2.0, 3.0, 3.0, 2.0",1.67,0.00,0.07,0.00,0.30,0.37,0.00,3.67,4.53,13.47,10.33,2.83,1.00,1791630.00,2.833333
Jamie Vardy,"3.0, 4.0, 3.0, 2.0, 2.0, 4.0",6.33,0.67,0.54,0.33,0.17,0.71,1.00,26.67,30.47,19.10,29.33,7.90,1.00,1204226.67,3.000000
Raúl Jiménez,"2.0, 2.0, 4.0, 2.0, 5.0, 5.0",3.67,0.33,0.43,0.00,0.08,0.51,0.33,14.00,17.67,14.40,35.67,6.80,1.00,992097.33,3.333333
Jørgen Strand Larsen,"2.0, 3.0, 2.0, 2.0, 2.0, 2.0",6.00,0.67,0.48,0.00,0.01,0.49,1.67,26.33,32.20,11.33,36.33,7.97,1.00,155986.33,2.166667


In [8]:
preferred_low = ["Average Difficulty"]

In [9]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df.data[weights.keys()]), columns=weights.keys())

# Step 2: Adjust columns where lower values are preferred by inverting their scale
for column in preferred_low:
    if column in df_normalized.columns:
        df_normalized[column] = 1 - df_normalized[column]  # Inverts normalized values so that lower is better

df_normalized.index = df.data.index
df_normalized


NameError: name 'df' is not defined

In [17]:
weights = {
    'Average Difficulty':-2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':0,
    'Expected Assists':0, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':10
}
df = calculate_weighted_score(forward_scores.data.drop('Difficulty', axis=1), weights, preferred_low)
df

,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Chris Wood,9.00,1.33,0.65,0.00,0.01,0.66,1.67,42.33,45.60,10.67,43.33,9.97,1.00,1656595.33,3.50,17.15
Jørgen Strand Larsen,6.00,0.67,0.48,0.00,0.01,0.49,1.67,26.33,32.20,11.33,36.33,7.97,1.00,155986.33,2.17,16.23
Alexander Isak,5.67,0.67,0.77,0.00,0.11,0.88,1.00,22.33,28.87,14.13,53.00,9.60,1.00,1356035.67,2.67,15.94
Danny Welbeck,7.00,0.67,0.44,0.33,0.01,0.45,1.33,27.67,29.93,10.70,27.67,6.83,1.00,2104154.67,2.83,15.10
Jamie Vardy,6.33,0.67,0.54,0.33,0.17,0.71,1.00,26.67,30.47,19.10,29.33,7.90,1.00,1204226.67,3.00,14.95
Dominic Solanke-Mitchell,6.67,0.67,0.54,0.33,0.01,0.55,1.00,24.67,27.53,5.13,28.00,6.07,1.00,1338234.33,3.00,14.80
Matheus Santos Carneiro Da Cunha,5.67,0.33,0.12,0.33,0.25,0.37,1.33,27.33,28.13,36.20,26.33,9.07,1.00,398022.67,2.17,14.30
Nicolas Jackson,5.33,0.67,0.32,0.00,0.03,0.35,1.33,25.00,28.33,15.70,21.00,6.50,1.00,1832263.33,3.00,14.10
Erling Haaland,3.33,0.33,1.07,0.00,0.01,1.08,0.00,10.33,12.47,5.27,56.00,7.37,1.00,7778840.67,3.33,13.87
Ollie Watkins,4.00,0.33,0.41,0.00,0.08,0.48,0.67,13.00,15.53,8.57,33.67,5.63,1.00,3048746.33,2.83,12.92


In [11]:
midfielders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==3)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
midfielders['difficulty'] = midfielders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
midfielders['avg_difficulty'] = midfielders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
midfielders['cost_per_point'] = midfielders['now_cost']/midfielders['total_points']
#forwards[:20].sort_values(by='difficulty')
#midfielders

In [19]:
mid_weights = {
    'Average Difficulty': -2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':1,
    'Expected Assists':1, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':1
}

midfielder_scores = compare_players(midfielders['full_name'], metrics=ATTACKER_METRICS, agg='mean')
midfielder_scores.data['Average Difficulty'] = midfielder_scores.data['Difficulty'].apply(lambda x: np.mean(x))
df = calculate_weighted_score(midfielder_scores.data.drop('Difficulty', axis=1), mid_weights, preferred_low)
df


,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Mohamed Salah,10.33,1.00,0.57,0.33,0.11,0.68,2.33,35.33,46.27,21.13,41.33,10.87,1.00,4230326.67,3.00,11.11
Bruno Borges Fernandes,5.67,0.33,0.62,0.33,0.14,0.76,1.00,28.00,34.07,42.53,27.33,10.37,1.00,722428.00,3.17,8.06
Morgan Rogers,5.33,0.67,0.49,0.00,0.10,0.59,0.67,16.33,24.33,16.27,21.00,6.17,1.00,2397337.33,2.83,7.11
Bryan Mbeumo,6.67,0.67,0.31,0.00,0.15,0.46,1.00,25.00,34.73,18.10,13.33,6.63,1.00,3694463.33,2.83,7.06
Antoine Semenyo,4.33,0.33,0.40,0.00,0.09,0.49,1.00,18.33,24.27,28.73,31.67,8.47,1.00,868745.67,2.50,6.65
Facundo Buonanotte,3.00,0.33,0.56,0.00,0.11,0.68,0.00,19.00,29.13,40.80,35.33,10.53,1.00,229313.00,3.00,5.97
Cole Palmer,5.00,0.33,0.22,0.00,0.24,0.46,1.33,22.67,18.80,25.33,17.67,6.20,1.00,5708525.33,3.00,5.85
Brennan Johnson,3.67,0.33,0.58,0.00,0.04,0.62,0.33,11.33,15.80,13.47,37.00,6.63,1.00,1054302.00,3.00,5.67
Alejandro Garnacho,4.67,0.33,0.36,0.00,0.03,0.39,1.00,18.33,21.47,19.23,56.67,9.67,1.00,927629.33,3.17,4.80
Youri Tielemans,3.67,0.00,0.08,0.33,0.22,0.30,1.00,20.67,24.40,45.90,10.67,8.10,1.00,160946.00,2.83,4.58


In [34]:
defenders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==2)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
defenders['difficulty'] = defenders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
defenders['avg_difficulty'] = defenders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
defenders['cost_per_point'] = defenders['now_cost']/defenders['total_points']
defender_scores = compare_players(defenders['full_name'], metrics=DEFENDER_METRICS, agg='mean')
defender_scores.data['Average Difficulty'] = defender_scores.data['Difficulty'].apply(lambda x: np.mean(x))


In [54]:
def_weights ={
 'Average Difficulty': -2,
 'Total Points': 3,
 'Clean Sheets': 1,
 'Goals Conceded': -1,
 'Own Goals': -1,
 'Yellow Cards': -1,
 'Bonus': 1.5,
 'Bps': 1,
 'Influence': 1,
 'Creativity': 1,
 'Threat': 1,
 'Ict Index': 1,
 'Starts': 1,
 'Expected Goal Involvements': 1.5,
 'Expected Goals Conceded': -1.5
 }

df = calculate_weighted_score(defender_scores.data.drop('Difficulty', axis=1), def_weights)
df

,Total Points,Minutes,Clean Sheets,Goals Conceded,Own Goals,Yellow Cards,Red Cards,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Expected Goal Involvements,Expected Goals Conceded,Average Difficulty,total_score
Joško Gvardiol,8.67,90.00,0.33,1.00,0.00,0.00,0.00,1.67,27.67,45.53,37.03,18.67,10.13,1.00,0.18,1.07,3.33,9.24
Milos Kerkez,6.33,86.67,0.33,0.67,0.00,0.00,0.00,1.00,25.67,27.20,24.50,5.67,5.73,1.00,0.20,1.35,2.50,7.00
Ola Aina,7.67,89.33,0.67,0.33,0.00,0.00,0.00,1.00,29.67,27.27,8.60,4.33,4.00,1.00,0.05,0.67,3.50,5.87
Lewis Hall,4.33,89.33,0.33,1.00,0.00,0.33,0.00,0.67,22.00,32.87,34.07,2.00,6.93,1.00,0.30,1.27,2.67,5.78
Murillo Santiago Costa dos Santos,5.33,90.00,0.67,0.33,0.00,0.00,0.00,0.67,22.33,22.67,11.63,10.67,4.50,1.00,0.13,0.67,3.50,5.15
Marc Guéhi,5.00,90.00,0.33,1.00,0.00,0.00,0.00,0.00,18.00,32.40,1.23,19.00,5.23,1.00,0.27,1.29,3.17,4.69
Ashley Young,4.67,90.00,0.33,0.67,0.00,0.00,0.00,0.33,18.67,23.27,22.57,3.67,4.97,1.00,0.23,1.08,3.17,4.51
Leif Davis,4.33,90.00,0.00,2.33,0.00,0.33,0.00,0.67,12.67,30.73,38.13,8.00,7.70,1.00,0.29,2.47,2.50,4.03
Michael Keane,5.67,90.00,0.33,0.67,0.00,0.67,0.00,1.00,17.33,30.40,3.67,14.00,4.83,1.00,0.06,1.08,3.17,3.75
Lucas Digne,3.67,83.33,0.33,1.67,0.00,0.67,0.00,0.67,13.33,17.20,28.37,10.67,5.63,1.00,0.28,1.47,2.83,3.63
